In [51]:
import time
import requests
import numpy as np
import pandas as pd
import re
!pip install scrapy
from scrapy.http import TextResponse





# **Staff.am Jobs scraping** 

In [52]:
URL="https://staff.am/en/jobs"

In [53]:
page=requests.get(URL)

In [54]:
type(page)

requests.models.Response

In [55]:
response= TextResponse(body=page.text,url=URL,encoding="utf-8")

In [56]:
def request(url):
    page = requests.get(url)
    response = TextResponse(body=page.text,url=url,encoding="utf-8")
    return response

In [57]:
def jobsto_scrape(url):
    page = requests.get(url)
    response= TextResponse(body=page.text,url=url,encoding="utf-8")
    
    vacancy_name=response.css("div.job-inner.job-item-title").extract()
    vacancy=[i.css('p.font_bold::text').extract_first() for i in response.css("div.job-inner.job-item-title")]
    company_name=response.css("div.job-inner.job-item-title").extract()
    company=[i.css('p.font_bold ~ p.job_list_company_title::text').extract_first() for i in response.css("div.job-inner.job-item-title")]
    base_url2 = "https://staff.am"
    job_page_URL = response.css("div.web_item_card.hs_job_list_item>a::attr(href)").extract()
    job_page_URL2 = [base_url2 + i for i in job_page_URL ]
    job_location = response.css("div[class = 'job-inner job-list-deadline'] >p[class='job_location']")
    job_loc = [i.css("::text").extract()[1] for i in job_location]
    location = [i.replace('\n',"").strip() for i in job_loc]
    deadline = response.css("div[class = 'job-inner job-list-deadline'] >p:not([class='job_location'])")
    post_deadline2 =[i.css('::text').extract()[1] for i in deadline]
    posting_deadline =[i.replace('\n'," ") for i in post_deadline2]
    return pd.DataFrame({"vacancy":vacancy,"company":company,'job_page_URL2':job_page_URL2,"posting_deadline":posting_deadline,'location':location})
staff_jobs = []
for i in range(1,20):
    current_page =jobsto_scrape(url = f"https://staff.am/en/jobs?page={i}&per-page=50")
    if current_page.shape[0] == 0:
        break
    else:
        staff_jobs.append(current_page)
staff_jobs = pd.concat(staff_jobs)
staff_jobs

    

,vacancy,company,job_page_URL2,posting_deadline,location
0,Creative Account Manager,Publicis Hepta LLC,https://staff.am/en/creative-account-manager-1,15 July 2020,Yerevan
1,Graphic Designer,Publicis Hepta LLC,https://staff.am/en/graphic-designer-376,15 July 2020,Yerevan
2,QA Engineer,PicsArt,https://staff.am/en/qa-engineer-509,31 July 2020,Yerevan
3,Web Designer,PicsArt,https://staff.am/en/web-designer-73,31 July 2020,Yerevan
4,Product Designer,PicsArt,https://staff.am/en/product-designer-26,31 July 2020,Yerevan
...,...,...,...,...,...
42,"Senior Software Engineer, Front-End",ServiceTitan,https://staff.am/en/sr-software-engineer-front...,03 July 2020,Yerevan
43,Senior Site Reliability Engineer,ServiceTitan,https://staff.am/en/senior-site-reliability-en...,15 July 2020,Yerevan
44,Lead Software Engineer,ServiceTitan,https://staff.am/en/lead-software-engineer,05 July 2020,Yerevan
45,QA Automation Engineer,ServiceTitan,https://staff.am/en/qa-automation-engineer-91,05 July 2020,Yerevan


 

*   **The number of times the specified company appears( current job posting count)**




In [58]:
staff_jobs['company']

0     Publicis Hepta LLC
1     Publicis Hepta LLC
2                PicsArt
3                PicsArt
4                PicsArt
             ...        
42          ServiceTitan
43          ServiceTitan
44          ServiceTitan
45          ServiceTitan
46          ServiceTitan
Name: company, Length: 547, dtype: object

In [59]:
staff_jobs['company'].count

<bound method Series.count of 0     Publicis Hepta LLC
1     Publicis Hepta LLC
2                PicsArt
3                PicsArt
4                PicsArt
             ...        
42          ServiceTitan
43          ServiceTitan
44          ServiceTitan
45          ServiceTitan
46          ServiceTitan
Name: company, Length: 547, dtype: object>

In [60]:
staff_jobs['company'].value_counts()

Digitain              32
SoftConstruct         23
PicsArt               21
ServiceTitan          16
TeamViewer Armenia    12
                      ..
AOByte                 1
Genacvale Tavern       1
VG LLC                 1
ANITEX LLC             1
LSOFT LTD              1
Name: company, Length: 230, dtype: int64



*   **The number of jobs posted by that companies**






1.   Digitain- 32
2.   SoftConstruct-23

1.   Picsart-21
2.   ServiceTitan-16 









*   **Jobs including word 'data' inside**




In [61]:
[staff_jobs['vacancy']]

[0                Creative Account Manager
 1                        Graphic Designer
 2                             QA Engineer
 3                            Web Designer
 4                        Product Designer
                      ...                 
 42    Senior Software Engineer, Front-End
 43       Senior Site Reliability Engineer
 44                 Lead Software Engineer
 45                 QA Automation Engineer
 46             Operations Center Engineer
 Name: vacancy, Length: 547, dtype: object]

In [62]:
staff_jobs1=staff_jobs[staff_jobs['vacancy'].str.contains("Data")]
print (staff_jobs1)

                                           vacancy  ... location
14                                  Data Scientist  ...  Yerevan
15                           Data Engineer (Spark)  ...  Yerevan
10                                    Data analyst  ...  Yerevan
34                                   Data Engineer  ...  Yerevan
19                   Junior Database Administrator  ...  Yerevan
42                   Senior Database Administrator  ...  Yerevan
16                   Data Specialist / QA Engineer  ...  Yerevan
29                          Database Administrator  ...  Yerevan
40  Engineering Manager, Java Microservices & Data  ...  Yerevan

[9 rows x 5 columns]


In [63]:
(staff_jobs1['vacancy']).count()

9

*9 jobs have the word "Data" inside* ( it was 9 when i checked the last time)
Data Scientist  
Data Engineer (Spark)  
Data analyst  
Data Engineer  
Junior Database Administrator  
Senior Database Administrator  
Data Specialist / QA Engineer  
Database Administrator  
Engineering Manager, Java Microservices & Data